# Предобработка

In [ ]:
import ...

In [ ]:
df = pd.read_csv(r'', sep = ';')

In [ ]:
#female = pd.read_excel(r'')

In [ ]:
#female

In [ ]:
#f = female['client_id'].to_list()

In [ ]:
#df = df.query('client_id in @f')

In [ ]:
#df

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

In [ ]:
df['date'] = df['date'].astype('str')

In [ ]:
df['date'] = df['date'].map(lambda x: dt.datetime.strptime (x, '%Y-%m-%d')) 

In [ ]:
#df['date'] = df['date'].map(lambda x: dt.datetime.strptime (x, '%Y%m%d')) # менять взависимости от всех данных

In [ ]:
#df = df.query('gs_with_disc > 19')

In [ ]:
df['client_id'] = df['client_id'].astype('int64')
df['pcs'] = df['pcs'].astype('int64')
df['gs_with_disc'] = df['gs_with_disc'].astype('int64')
df['gs_without_disc'] = df['gs_without_disc'].astype('int64')
df['trans_numb'] = df['trans_numb'].astype('int64')

In [ ]:
df.info()

In [ ]:
df['gs_with_disc'].describe(percentiles = [0.015,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.985,]).apply("{0:.5f}".format)

In [ ]:
#print(df['gs_with_disc'].mean() - 3 * df['gs_with_disc'].std(), df['gs_with_disc'].mean() + 3 * df['gs_with_disc'].std())

In [ ]:
frod = df.query('gs_with_disc > 5200') #нужно поправить после всей выборки

In [ ]:
# здесь надо вписать нижний и верхний перцентиль
data_new = df.query('gs_with_disc <= 5200 ') #нужно поправить после всей выборки

In [ ]:
data_new 

In [ ]:
data_new['gs_with_disc'].describe(percentiles = [0.015,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.985]).apply("{0:.5f}".format)

# Расчеты для RFM

In [ ]:
data_new = data_new.reset_index()

In [ ]:
data_new = data_new.drop(columns = 'index')

In [ ]:
data_new

In [ ]:
 data_new =  data_new.groupby(['date', 'client_id']).sum().reset_index()

In [ ]:
 data_new 

In [ ]:
# сколько дней от текущей даты прошло (конца года) с момента последней покупки 
req = data_new.groupby('client_id')['date'].max()
req = pd.DataFrame(req).reset_index()
req['req'] = pd.to_datetime('2023-05-28') - req['date']
req = req.sort_values(by = 'req')
req = req.rename(columns={"date": "last_purch_date", "req": "recency"})

In [ ]:
def recency(recency):

    res = str(recency).split(' ')[0]

    return(int(res))


req['recency'] = req['recency'].apply(recency)

In [ ]:
req

In [ ]:
re2=req.query('recency>0')

In [ ]:
re2['recency'].mean()

In [ ]:
req['recency'].mean()

In [ ]:
# сколько денег принес клиент
monetary = data_new.groupby('client_id')['gs_with_disc'].sum() 
monetary= pd.DataFrame(monetary).reset_index()

In [ ]:
monetary = monetary.sort_values(by = 'gs_with_disc', ascending = False)
monetary  = monetary.rename(columns={"gs_with_disc": "monetary"})
monetary

In [ ]:
# жизненный цикл клиента
lifetime_1 = data_new.groupby('client_id')['date'].max()
lifetime_1= pd.DataFrame(lifetime_1).reset_index()
lifetime_2 = data_new.groupby('client_id')['date'].min()
lifetime_2= pd.DataFrame(lifetime_2).reset_index()
lifetime = lifetime_1.merge(lifetime_2, left_on='client_id', right_on='client_id')
lifetime['lifetime'] = lifetime ['date_x'] - lifetime ['date_y']
lifetime = lifetime.sort_values(by = 'lifetime', ascending = False)
lifetime  = lifetime.rename(columns={"date_x": "last_pur_date", "date_y" : "first_pur_date"})
lifetime

In [ ]:
def recency(recency):

    res = str(recency).split(' ')[0]

    return(int(res))


lifetime['lifetime'] = lifetime['lifetime'].apply(recency)

In [ ]:
lifetime

In [ ]:
# сколько было покупок за год
freq = data_new.groupby('client_id')['date'].count()
freq= pd.DataFrame(freq).reset_index()
freq

In [ ]:
freq =  freq.sort_values(by = 'date', ascending = False)
freq ['freq'] = lifetime ['lifetime'] / freq ['date'] 
freq  = freq.rename(columns={"date": "amount_purch", "freq" : "frequency"})
freq

In [ ]:
freq['amount_purch'].describe(percentiles = [0.015,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.985]).apply("{0:.5f}".format)

In [ ]:
#freq = freq.query('amount_purch < 10')#нужно поправить после всей выборки
freq= pd.DataFrame(freq).reset_index()
freq = freq.drop(columns = 'index')
freq

In [ ]:
#frod2= freq.query('amount_purch > 10') #нужно поправить после всей выборки

In [ ]:
rfm = freq.merge(lifetime, how = 'left', left_on='client_id', right_on='client_id')
rfm = rfm.merge(monetary, how = 'left', left_on='client_id', right_on='client_id')
rfm = rfm.merge(req, how = 'left', left_on='client_id', right_on='client_id')
rfm = rfm.drop(columns = 'last_purch_date')
rfm['ATV'] = rfm['monetary']/rfm['amount_purch']

In [ ]:
rfm

In [ ]:
rfm = rfm.query('last_pur_date < "2021-06-01"')

# Кластеризация

In [ ]:
rfm_new = rfm.drop(columns = ['frequency', 'ATV', 'last_pur_date', 'first_pur_date']) 

In [ ]:
rfm_new

In [ ]:
def frequency(rfm_new):
    frequency = []
    for i in range (len(rfm_new)):
        if rfm_new.iloc[i,1] == 1 :
            frequency.append(rfm_new.iloc[i,4]/rfm_new.iloc[i,1])
        else:
            frequency.append(rfm_new.iloc[i,2]/rfm_new.iloc[i,1])
    rfm_new['frequency'] = frequency      

In [ ]:
frequency(rfm_new)
rfm_new

In [ ]:
rfm_new

In [ ]:
#frod2 #нужно поправить после всей выборки

In [ ]:
clasters = rfm_new

In [ ]:
frod 

In [ ]:
rfm_new['amount_purch'].describe(percentiles = [0.015,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.985]).apply("{0:.5f}".format)

In [ ]:
rfm_new = rfm_new.query('amount_purch<=34')

In [ ]:
rfm_new

In [ ]:
rfm

In [ ]:
rfm.info()

In [ ]:
rfm['year'] = rfm['last_pur_date'].dt.year

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


plt.figure(figsize=(15, 10))

sns.boxplot(x='year', y='recency', data=rfm)

plt.title('Recency Boxplot by Year')
plt.xlabel('Year')
plt.ylabel('Recency')

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 10))
sns.boxplot(y='recency', data=rfm)

plt.title('Распределение recency для клиентов')
plt.ylabel('Recency')

plt.ylim(0, rfm['recency'].quantile(0.95))

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))

sns.histplot(x='recency', data=rfm, color='steelblue', alpha=0.7)

plt.title('Recency Histogram')
plt.xlabel('Recency')
plt.ylabel('Count')

plt.show()


In [ ]:

mean = rfm['recency'].mean()
std = rfm['recency'].std()

data = np.random.normal(mean, std, size=len(rfm))

plt.figure(figsize=(15, 10))
plt.hist(data, bins=50, color='steelblue', alpha=0.7)

plt.title('Стандартное распределение Recency')
plt.xlabel('Recency')
plt.ylabel('кол-во')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 10))
plt.boxplot(rfm['lifetime'], whis=[25,85])


plt.title('Боксплот для Lifetime')
plt.ylabel('Lifetime')

plt.show()


In [ ]:
rfm_new['recency'].describe(percentiles = [0.015,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.985]).apply("{0:.5f}".format)

In [ ]:
rfm_new=rfm_new.query('recency<=1539')

In [ ]:
 data_new

In [ ]:
plt.figure(figsize=(15, 10))

grouped_data = rfm.groupby('amount_purch')['client_id'].nunique().reset_index()


sns.barplot(x='amount_purch', y='client_id', data=grouped_data, color='steelblue')

plt.title('Гистограмма количества покупок и клиентов')
plt.xlabel('Количество покупок (amount_purch)')
plt.ylabel('Количество клиентов (client_id)')

plt.show()



In [ ]:
single_purchase_clients = rfm[rfm['amount_purch'] == 1]

plt.figure(figsize=(15, 10))
sns.histplot(x='recency', data=single_purchase_clients, color='steelblue', alpha=0.7)

plt.title('Распределение Recency для клиентов с одной покупкой')
plt.xlabel('Recency')
plt.ylabel('Количество клиентов')




In [ ]:
rfm_new

In [ ]:
rfm_new.to_csv(r'')